You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__ are currently saved in `office_addresses.csv`. If the value for office is `NaN`, then the employee is remote. 
- __Employee addresses__ are saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ are saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. However, this sheet was edited at some point, and the headers were removed! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ have been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}

In [188]:
import pandas as pd
# Start coding here... 
#look at the office_add file 
df = pd.read_csv('datasets/office_addresses.csv')
#df.shape - 3 rows, 5 columns
#df.info() - check the table info
display(df)

,office,office_country,office_city,office_street,office_street_number
0,Leuven Office,BE,Leuven,Martelarenlaan,38
1,ESB Office,US,New York City,Fifth Avenue,350
2,WeWork Office,GB,London,Old Street,207


In [189]:
# load the employee info as df1 (tab 1)
df1 = pd.read_excel('datasets/employee_information.xlsx')
# df1.shape 4 rows, 7 columns
# df1.info() 4 non-null
display(df1)

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7


In [190]:
# load the employee info as df2 (tab 2)

df2 = pd.read_excel('datasets/employee_information.xlsx', sheet_name = 'emergency_contacts', header = None)
# df2.shape 3 rows 6 columns
# df2.info() 3 non null

# create list for columns name
columns = ['employee_id', 'last_name', 'first_name', 'emergency_contact', 'emergency_contact_number', 'relationship']
df2.columns = columns
display(df2)




,employee_id,last_name,first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


In [191]:
#load the employee role json file as df3
df3 = pd.read_json('datasets/employee_roles.json')
# df3.shape 3 rows 4 columns
# df3.info() 3 non-null
df3_T = df3.transpose()
df3_T.index.name = 'employee_id'
display(df3_T)

,title,monthly_salary,team
employee_id,,,
A2R5H9,CEO,$4500,Leadership
H8K0L6,CFO,$4500,Leadership
G4R7V0,Business Developer,$3000,Sales
M1Z7U9,Office Manager,$2000,People Operations


In [192]:
# merge the offices (df) and address together(df1)
info = df1.merge(df,left_on='employee_country',right_on= 'office_country',how='left')

#replace missing values in column name starting with office with "remote"
office_info = info.fillna('Remote')
display(office_info)

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number,office,office_country,office_city,office_street,office_street_number
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,Leuven Office,BE,Leuven,Martelarenlaan,38.0
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221,WeWork Office,GB,London,Old Street,207.0
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,ESB Office,US,New York City,Fifth Avenue,350.0
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Remote,Remote,Remote,Remote,Remote


In [193]:
#merge the employee info(df1) with their roles(df3_T)
emp = df1.merge(df3_T,on ='employee_id',how='inner')
display(emp)

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number,title,monthly_salary,team
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,CEO,$4500,Leadership
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221,CFO,$4500,Leadership
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,Business Developer,$3000,Sales
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Office Manager,$2000,People Operations


In [194]:
# Merge the employees info(emp) with their emergency contact(df2)
emp_info = emp.merge(df2, left_on ='employee_id' , right_on ='employee_id' , how='left')
display(emp_info)

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number,title,monthly_salary,team,last_name,first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,CEO,$4500,Leadership,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221,CFO,$4500,Leadership,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,Business Developer,$3000,Sales,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Office Manager,$2000,People Operations,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


In [195]:
employees_temp.columns

Index(['employee_id', 'last_name', 'first_name', 'employee_country',
       'employee_city', 'employee_street', 'employee_street_number', 'title',
       'monthly_salary', 'team', 'emergency_contact',
       'emergency_contact_number', 'relationship', 'office', 'office_country',
       'office_city', 'office_street', 'office_street_number'],
      dtype='object')

In [196]:

# Merge emp_info table with the office_info table using 'employee_id' as the key
employees_temp = emp_info.merge(office_info, on='employee_id', how='outer', suffixes=('_x', '_y'))

# Drop columns from employees_temp that have the suffix '_y'
employees_temp.drop(employees_temp.filter(regex='_y$').columns, axis=1, inplace=True)
# Drop extra columns
Columns =['first_name', 'last_name']
employees_temp.drop(columns = Columns,inplace=True)
#Re-name the columns
employees_temp = employees_temp.rename(columns={
    'employee_first_name_x': 'first_name',
    'employee_last_name_x': 'last_name',
    'employee_country_x': 'employee_country',
    'employee_city_x': 'employee_city',
    'employee_street_x': 'employee_street',
    'employee_street_number_x': 'employee_street_number'
})


#Re-order the columns
final_columns = ["employee_id", "first_name", "last_name", "employee_country", 
                 "employee_city", "employee_street", "employee_street_number", 
                 "emergency_contact", "emergency_contact_number", "relationship", 
                 "monthly_salary", "team", "title", "office", "office_country", 
                 "office_city", "office_street", "office_street_number"]
employees_final = employees_temp.reindex(columns= final_columns)

#set index
employees_final = employees_final.set_index('employee_id')
# Display the resulting DataFrame after dropping columns
display(employees_final)





,first_name,last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,$4500,Leadership,CEO,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,$4500,Leadership,CFO,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,$3000,Sales,Business Developer,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,$2000,People Operations,Office Manager,Remote,Remote,Remote,Remote,Remote
